# FourD.js Rewrite

[Joshua Marshall Moore](mailto:moore.joshua@pm.me)

June 1st, 2018

It's been years since I've worked on the core FourD.js. Since then, Javascript has added a few features, and it's time to incorporate them into FourD.js. 

First, we require THREE.js, nowadays conveniently available as an NPM package. 

In [29]:
var THREE = require('three');

Next, let's define some constants:

In [30]:
var CONSTANTS = {
  scene: {
    width: 1000,
    height: 500,
    far: 100000,
    zoom: -50,
    antialias: true,
    background: 0xffffff,
    camera_distance: -50,
  },
  vertex: {
    size: 10,
    color: 0x000000,
    wireframe: false,
    texture: undefined
  },
  edge: {
    strength: 1.0
  },
  bhn3: {
    inner_distance: 0.36
  },
  layout: {
    epsilon: 0.1,
    attraction: 0.1,
    repulsion: 25.0,
    friction: 0.60,
  }
};

The Cube class encapsulates everything needed to draw a cube. I used to achieve this using composition, but now I have inheritance. 

In [31]:
class Cube extends THREE.Mesh{
  constructor(options){    
    var options = options || {
      size: CONSTANTS.vertex.size,
      color: CONSTANTS.vertex.color,
      wireframe: CONSTANTS.vertex.wireframe,
      texture: CONSTANTS.vertex.texture
    };

    var geometry = new THREE.BoxGeometry(
      options.size,
      options.size,
      options.size
    );
    geometry.dynamic = true;
    
    var material_args;
    if(options.texture !== undefined){
      material_args = {
        map: new THREE.TextureLoader().load(options.texture)
      };
    }else{
      material_args = {
        color: options.color,
        wireframe: options.wireframe
      }
    }
    
    var material = new THREE.MeshBasicMaterial(material_args);
    
    super(geometry, material);

    this.matrixAutoUpdate = true;
    
    this.options = options;
    return this;
  }
}

In [32]:
var c = new Cube();
console.log(c.position)

Vector3 { x: 0, y: 0, z: 0 }


In [33]:
class Vertex extends THREE.Group{
  constructor(options){
    super();

    this.position.set(
      Math.random(),
      Math.random(),
      Math.random()
    );
    
    this.velocity = new THREE.Vector3();
    this.acceleration = new THREE.Vector3();
    
    var options = options || {};
    
    if(options.cube){
      var cube = new Cube(options.cube);
      this.add(cube);
      cube.vertex = this;
    }
    
    if(options.label){
      var label = new Label(options.label);
      this.add(label);
      label.vertex = this;
    }
    
    this.edges = new Set();
    return this;
  }
  
  psint(scene){
    scene.add(this);
  }
}

In [34]:
var v = new Vertex();
console.log(v.id);

4


In [35]:
var v = new Vertex();
console.log(v.id);

5


In [36]:
class Edge extends THREE.Line{
  constructor(source, target, options){
    var options = options || {
      color: 0x000000,
      transparent: false,
      opacity: 1.0,
      attraction: CONSTANTS.edge.attraction
    }
    
    var geometry = new THREE.Geometry();
    geometry.dynamic = true;
    geometry.vertices.push(source.position);
    geometry.vertices.push(target.position);
    geometry.verticesNeedUpdate = true;
    
    var material = new THREE.LineBasicMaterial(options);
    
    super(geometry, material);
    this.source = source;
    source.edges.add(this);
    this.target = target;
    target.edges.add(this);
    this.frustumCulled = false;

    return this;
  }
  
  prepare_destruction(){
    this.source.edges.delete(this);
    this.target.edges.delete(this);
  }
  
  paint(scene){
    scene.add(this);
  }
}

In [37]:
class BarnesHutNode3{
  constructor(constants){
    this.inners = new Set();
    this.outers = new Map();
    this.center_sum = new THREE.Vector3();
    
    this.constants = constants || {
      inner_distance: CONSTANTS.bhn3.inner_distance,
      repulsion: CONSTANTS.bhn3.repulsion
    }
  }
  
  center(){
    return this.center_sum.clone().divideScalar(this.inners.size);
  }
  
  place_inner(vertex){
    this.inners.add(vertex);
    this.center_sum.add(vertex.position);
  }
  
  get_octant(position){
    var center = this.center();
    var x = center.x < position.x ? 'l' : 'r';
    var y = center.y < position.y ? 'u' : 'd';
    var z = center.z < position.z ? 'i' : 'o';
    return x + y + z;
  }
  
  place_outer(vertex){
    var octant = this.get_octant(vertex.position);
    if(!this.outers.has(octant)){
      this.outers.set(octant, new BarnesHutNode3);
    }
    this.outers.get(octant).insert(vertex);
  }
  
  insert(vertex){
    if(!this.inners.size){
      this.place_inner(vertex);
    }else{
      if(this.center().distanceTo(vertex.position) <= this.constants.inner_distance){
        this.place_inner(vertex);
      }else{
        this.place_outer(vertex);
      }
    }
  }
  
  estimate(vertex, force, force_fn){
    if(this.inners.has(vertex)){
      this.inners.delete(vertex);
      
      this.inners.forEach(v => {
        force.add(force_fn(vertex.position, v.position));
      });
      
      this.inners.add(vertex);
    }else{
      var sumstimate = force_fn(vertex.position, this.center());
      sumstimate.multiplyScalar(this.inners.size);
      force.add(sumstimate);
    }
    
    this.outers.forEach(node => node.estimate(vertex, force, force_fn));
  }
  
  static pairwise_repulsion(x1, x2){
    console.assert(x1);
    console.assert(x2);
    
    x1 = x1.clone();
    x2 = x2.clone();

    var difference = x1.sub(x2);
    
    // first term
    var enumerator1 = CONSTANTS.layout.repulsion;
    var abs_difference = difference.length();
    
    var sum = CONSTANTS.layout.epsilon + abs_difference;
    var denominator1 = Math.pow(sum, 2);
    
    var term1 = enumerator1/denominator1;
    
    // second term
    var enumerator2 = difference;
    var denominator2 = abs_difference;
    
    var term2 = enumerator2.divideScalar(denominator2);
    
    return term2.multiplyScalar(term1);
  }
}

In [38]:
class Graph extends THREE.Scene{
  constructor(constants){
    super();
    this.V = new Set();
    this.E = new Set();
    this.constants = constants || {
      attraction: CONSTANTS.layout.attraction,
      repulsion: CONSTANTS.layout.repulsion,
      friction: CONSTANTS.layout.friction
    }
  }
  
  clear(){
    this.V = new Set();
    this.E = new Set();
  }
  
  add_vertex(options){
    var vertex = new Vertex(options);
    this.V.add(vertex);
    this.add(vertex);
    return vertex;
  }
    
  add_edge(source, target, options){
    var edge = new Edge(source, target, options);
    this.E.add(edge);
    this.add(edge);
    return edge;
  }
  
  remove_edge(edge){
    edge.prepare_destruction();
    this.remove(edge);
    this.E.delete(edge);
  }
  
  remove_vertex(vertex){
    vertex.edges.forEach(edge => {
      this.remove_edge(edge);
    });
    
    this.remove(vertex);
    this.V.delete(vertex);
  }
  
  layout(){
    var tree = new BarnesHutNode3();
    
    this.V.forEach(vertex => {
      vertex.acceleration = new THREE.Vector3();
      vertex.repulsion_forces = new THREE.Vector3();
      vertex.attraction_forces = new THREE.Vector3();
      
      tree.insert(vertex);
    });
    
    this.V.forEach(vertex => {
      tree.estimate(
        vertex, 
        vertex.repulsion_forces, 
        BarnesHutNode3.pairwise_repulsion
      );
    });
    
    // calculate attractions
    this.E.forEach(edge => {
      var f = edge.source.position.clone().sub(edge.target.position);
      f.multiplyScalar(-1 * this.constants.attraction * edge.strength);
      edge.source.attraction_forces.sub(f);
      edge.target.attraction_forces.add(f);
      
      // this is probably the place for gravity
    });
    
    this.V.forEach(vertex => {
      var friction = vertex.velocity.multiplyScalar(this.constants.friction);
      
      vertex.acceleration.add(
        vertex.repulsion_forces.clone().add(
          vertex.attraction_forces.clone().negate()
        )
      );
      vertex.acceleration.sub(friction);
      vertex.velocity.add(vertex.acceleration);
      vertex.position.add(vertex.velocity);
    });
    
    this.E.forEach(edge => {
      edge.geometry.dirty = true;
      edge.geometry.verticesNeedUpdate = true;
    });
  }
}

In [39]:
var g = new Graph();
var v1 = g.add_vertex();
var v2 = g.add_vertex();
var e = g.add_edge(v1, v2)
console.log(v1.position, v2.position);

g.layout();
console.log(v1.position, v2.position);

THREE.Material: 'attraction' parameter is undefined.


Vector3 {
  x: 0.5691243899291494,
  y: 0.7417435251059459,
  z: 0.16790771344409716 } Vector3 {
  x: 0.8868123454880015,
  y: 0.7877323383790213,
  z: 0.20499422423826452 }
Vector3 {
  x: -136.70856682947232,
  y: -19.130706775826006,
  z: -15.857725264207662 } Vector3 {
  x: 138.1645035648895,
  y: 20.660182639310975,
  z: 16.230627201890023 }


In [40]:
var width = 500;
var height = 300;

var scene = new THREE.Scene();
var camera = new THREE.PerspectiveCamera( 75, width/height, 0.1, 1000 );

$$html$$ = `<canvas width="${width}" height="${height}"></canvas>`
var renderer = new THREE.WebGLRenderer({canvas: $$html$$});
renderer.setSize( width, height );

In [41]:
class FourD{
  constructor(options){
    this.options = options || {
      width: CONSTANTS.scene.width,
      height: CONSTANTS.scene.height,
      canvas: undefined,
      far: CONSTANTS.scene.far,
      antialias: CONSTANTS.scene.antialias,
      background: CONSTANTS.scene.background,
      camera_distance: CONSTANTS.scene.camera_distance,
    };
    
    this.graph = new Graph(); // Graph inherits from Scene
    this.scene = this.graph;
    
    this.camera = new THREE.PerspectiveCamera(
      75, 
      this.options.width / this.options.height,
      1,
      this.options.far
    );
    
    this.light = new THREE.PointLight(0xf0f0f0);
    this.scene.add(this.camera);
    this.scene.add(this.light);
    
    this.renderer = new THREE.WebGLRenderer({antialias: true});
    this.renderer.setClearColor(this.options.background);
    this.renderer.setSize(this.options.width, this.options.height);
    
    this.camera.position.z = this.options.camera_distance;
    this.camera.lookAt(new THREE.Vector3());
    
    this.clock = new THREE.Clock();
    this.controls = new THREE.OrbitControls(camera, this.options.canvas);
    this.controls.update(clock.getDelta());
    this.controls.movementSpeed = 250;
    this.controls.domElement = this.renderer.domElement;
    this.controls.rollSpeed = Math.PI / 12;
    this.controls.autoForward = false;
    this.controls.dragToLook = true;
    
    this.render();
    
    return this;
  }
  
  resolve_click(event){
    if(event.target === this.renderer.domElement){
      var raycaster = new THREE.Raycaster();
      mouse = new THREE.Vector2();
      mouse.x = ( event.clientX / this.renderer.domElement.width ) * 2 - 1;
      mouse.y = - ( event.clientY / this.renderer.domElement.height ) * 2 + 1;
      raycaster.setFromCamera(mouse, this.camera);
      intersects = raycaster.intersectObjects(this.scene.children, true);

      if(intersects.length > 0){
        return intersects[0].object.vertex;
      }else{
        return null;
      }
    }
  }
  
  render(){
    requestAnimationFrame(this.render);
    
    this.graph.layout();
    this.controls.update(this.clock.getDelta());
    
    this.renderer.render(this.graph, this.camera);
  }
}

THREE.WebGLRenderer 93
